In [1]:
import random
from collections import Counter

import numpy as np
import pandas as pd
from PIL import Image

# Блок с различными функциями которые потребуются далее (или нет).

Тут живут функции, которые ответственны за подсчет расстояния между картинками. Картинки бывают трех типов: **RGB, градация серого и чб**.

In [2]:
def distance_for_RGB(pix1, pix2):
    pixel_distance = 0

    for color_channel in range(3):
        pixel_distance += (abs(pix1[color_channel] - pix2[color_channel]) / 3)

    return pixel_distance


def distance_for_grey(pix1, pix2):
    return abs(pix1 - pix2)


def distance_for_bw(pix1, pix2):
    return int(pix1 != pix2)


def distance(img1: Image, img2: Image, images_type='RGB'):
    if img2.size != img1.size:
        raise RuntimeError(f"Size must be the same! {img1.size} != {img2.size}")

    distance_function = None
    img1 = img1.copy()
    img2 = img2.copy()
    img1 = img1.convert(images_type)
    img2 = img2.convert(images_type)

    if images_type == 'RGB':
        distance_function = distance_for_RGB
    elif images_type == 'L':
        distance_function = distance_for_grey
    elif images_type == '1':
        distance_function = distance_for_bw

    width, height = img2.size
    img1_pix = img1.load()
    img2_pix = img2.load()
    total_distance = 0

    for x in range(width):
        for y in range(height):
            if img1_pix[x, y] != img2_pix[x, y]:
                total_distance += distance_function(img1_pix[x, y], img2_pix[x, y])

    return total_distance

(⊙o⊙) Ищем соседей. Ну и считаем норм подобрали или шляпа какая-то.

In [3]:
def k_means(noised_image: Image):
    width, height = noised_image.size
    min_dist = 255 * 255 * 255 * width * height
    nearest_number = 0

    for n in range(10):
        number_reference = Image.open(f'../../resources/numbers/{width}x{height}/{n}.png')
        tmp_dist = distance(number_reference, noised_image)
        if min_dist > tmp_dist:
            nearest_number = n
            min_dist = tmp_dist

    return nearest_number


def predict(numbers, assumption_func):
    data = []
    wrong_data = []
    success_predicted = 0
    is_empty = True

    for number, number_values in numbers.items():
        wrong_predicted = []
        tmp = []

        for noise in number_values.values():
            wrong_assumption = []

            for image in noise:
                assumption = assumption_func(image)
                if number == assumption:
                    success_predicted += 1
                else:
                    is_empty = False
                    wrong_assumption.append(assumption)

            wrong_predicted.append(Counter(wrong_assumption))
            tmp.append(success_predicted)
            success_predicted = 0

        data.append(tmp)
        wrong_data.append(wrong_predicted)
    if is_empty:
        wrong_data = []
    return data, wrong_data


def odin(images_list: list, bound=4, number_of_occurrences=1):
    res = []
    for number in images_list:
        tmp_list = images_list.copy()
        tmp_list.remove(number)
        tmp_list.sort(key=lambda img: distance(number, img, img.mode))
        res.append(tmp_list[:bound])

    t = []
    for n in res:
        for number in n:
            t.append(number)

    out = 0
    for number in images_list:
        if t.count(number) <= number_of_occurrences:
            out += 1

    return out

Шумим картинки подходящим шумом.
Шумим равномерно, шумим область.
Шумим эталоны, чтоб в дальнейшем играть в угадайку.

In [4]:
def RGB_noise(pixel):
    res = tuple(np.random.choice(range(256), size=3))
    while res == pixel:
        res = tuple(np.random.choice(range(256), size=3))
    return res


def grey_noise(pixel):
    res = random.randint(0, 255)
    while res == pixel:
        res = random.randint(0, 255)
    return res


def bw_noise(pixel):
    if pixel == 0:
        return 255
    else:
        return 0


def noise_image(input_image: Image, probability: float):
    noise_func = None
    if input_image.mode == "RGB":
        noise_func = RGB_noise
    elif input_image.mode == "L":
        noise_func = grey_noise
    elif input_image.mode == "1":
        noise_func = bw_noise

    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)

    for x in range(width):
        for y in range(height):
            if random.random() < p:
                pixels[x, y] = noise_func(pixels[x, y])

    return input_image


def noise_image_area(input_image: Image, center: (int, int), radius: int, probability: float):
    noise_func = None
    if input_image.mode == "RGB":
        noise_func = RGB_noise
    elif input_image.mode == "L":
        noise_func = grey_noise
    elif input_image.mode == "1":
        noise_func = bw_noise

    input_image = input_image.copy()
    width, height = input_image.size
    pixels = input_image.load()
    p = random.uniform(0, probability)
    x_start = 0
    y_start = 0
    x_finish = width
    y_finish = height
    if center[0] - radius > 0:
        x_start = center[0] - radius
    if center[1] - radius > 0:
        y_start = center[1] - radius
    if center[0] + radius < width:
        x_finish = center[0] + radius
    if center[1] + radius < height:
        y_finish = center[1] + radius

    for x in range(x_start, x_finish):
        for y in range(y_start, y_finish):
            if random.random() < p:
                pixels[x, y] = noise_func(pixels[x, y])

    return input_image


def generate_noise_numbers(width, height, noise_func, tries_per_noise, image_type='RGB'):
    res = {}

    for n in range(10):
        number = Image.open(f'../../resources/numbers/{width}x{height}/{n}.png').convert(image_type)
        noise_dict = {}

        for noise in noise_indexes:
            noise_list = []

            for i in range(tries_per_noise):
                noised_number = noise_func(number, noise)
                noise_list.append(noised_number)

            noise_dict.update({noise: noise_list})
        res.update({n: noise_dict})

    return res


def generate_noise(mode, size, color, probability, noise_func):
    return noise_func(Image.new(mode, size, color), probability)

А это просто для вывода уже всего высчтаного. Ну и сравнение ради сравнения.

In [5]:
def draw_assumption(data):
    df = pd.DataFrame(data[0])
    df.columns = noise_indexes
    print(df)
    print('-' * 100)

    if data[1]:
        ejection = pd.DataFrame(data[1])
        ejection.columns = noise_indexes
        print(ejection)
        print("=" * 100)


def check_results(*data_list):
    equals = True
    if len(data_list) < 2:
        return
    for i in range(1, len(data_list)):
        if data_list[0] != data_list[i]:
            equals = False
    print("Equals =", equals)
    draw_assumption(data_list[0])
    if not equals:
        for i in range(1, len(data_list)):
            draw_assumption(data_list[i])

# Да будет МЛная магия!

Какой величины шум.

In [6]:
noise_indexes = list(np.arange(0.3, 0.65, 0.05))

Генерируем равномерно зашумленные эталончики. Каждый со своим "родным" шумом!

In [7]:
rgb_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_image(image, noise), 10, 'RGB')
grey_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_image(image, noise), 10, 'L')
bw_numbers = generate_noise_numbers(7, 8, lambda image, noise: noise_image(image, noise), 10, '1')

Тут мы уже **зашумленные RGB картинки** отгадываем разными вариантами. Стандартный, конвертим в градацию серого, конвертим в чб.

In [8]:
check_results(predict(rgb_numbers, lambda noised_number: k_means(noised_number)),
              predict(rgb_numbers, lambda noised_number: k_means(noised_number.convert('L'))),
              predict(rgb_numbers, lambda noised_number: k_means(noised_number.convert('1'))))

Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8   

Тут мы уже **зашумленные картинки в градации серого** отгадываем или стандартно, или конвертим перед этим в чб.

In [9]:
check_results(predict(grey_numbers, lambda noised_number: k_means(noised_number)),
              predict(grey_numbers, lambda noised_number: k_means(noised_number.convert('1'))))

Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10     9
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10     9    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8    10    10    10    10    10    10     9
9    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
  0.30 0.35 0.40 0.45 0.50    0.55    0.60
0   {}   {}   {}   {}   {}      {}  {9: 1}
1   {}   {}   {}   {}   {}      {}      {}
2   {}   {}   {}   {}   {}      {}      {}
3   {}   {}   {}   {}   {}      {}      {}
4   {}   {}   {}   {}   {}      {}      {}
5   {}   {}   {}   {}   {}  {3: 1}      {}
6   {}   {}   {}   {}   {}      {}      {}
7   {}   {}   {}   {}   {}      {}      {}
8   {}   {}  

Ну тут вариантов особо не осталось. Поэтому картинки зашумлены в чб, отгадываем в чб.

In [10]:
draw_assumption(predict(bw_numbers, lambda noised_number: k_means(noised_number)))

   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0     7     9     7     9     8     8     5
1    10    10     9     9     7     6     6
2     9     8    10     9     5     7     6
3     8     7    10     4     5     7     4
4    10     8     7     9     4     6     5
5    10     9     7     8     9     6     5
6    10     9     8     9     3     5     8
7    10    10     9     8     9     5     6
8     8     8     8     6     7     5     4
9     9    10     6     6     9     7     7
----------------------------------------------------------------------------------------------------
           0.30                0.35                0.40  \
0  {6: 2, 9: 1}              {6: 1}        {6: 2, 7: 1}   
1            {}                  {}              {7: 1}   
2        {7: 1}              {7: 2}                  {}   
3  {2: 1, 5: 1}  {1: 1, 7: 1, 2: 1}                  {}   
4            {}        {8: 1, 9: 1}  {7: 1, 3: 1, 1: 1}   
5            {}              {1: 1}        {6: 2, 1: 1}   
6 

А тут уже генерируем картинки с шумом в какой-то области. Центр выбираем случайно. Радиусов несколько, чтобы посмотреть как тут с устойчивостью.

In [11]:
x_rand = random.randint(0, 7)
y_rand = random.randint(0, 8)
radius_list = list(np.arange(3, 6, 1))
print(x_rand, y_rand, radius_list)

rgb_numbers_area = []
bw_numbers_area = []
grey_numbers_area = []
for radius in radius_list:
    rgb_numbers_area.append(
        generate_noise_numbers(7, 8, lambda image, noise: noise_image_area(image, (x_rand, y_rand), radius, noise), 10,
                               'RGB'))
    bw_numbers_area.append(
        generate_noise_numbers(7, 8, lambda image, noise: noise_image_area(image, (x_rand, y_rand), radius, noise), 10,
                               '1'))
    grey_numbers_area.append(
        generate_noise_numbers(7, 8, lambda image, noise: noise_image_area(image, (x_rand, y_rand), radius, noise), 10,
                               'L'))

5 8 [3, 4, 5]


Тут мы уже **зашумленные RGB картинки** отгадываем разными вариантами. Стандартный, конвертим в градацию серого, конвертим в чб.

In [12]:
for number in rgb_numbers_area:
    check_results(predict(number, lambda noised_number: k_means(noised_number)),
                  predict(number, lambda noised_number: k_means(noised_number.convert('L'))),
                  predict(number, lambda noised_number: k_means(noised_number.convert('1'))))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10   

Тут мы уже **зашумленные картинки в градации серого** отгадываем или стандартно, или конвертим перед этим в чб.

In [13]:
for number in grey_numbers_area:
    check_results(predict(number, lambda noised_number: k_means(noised_number)),
                  predict(number, lambda noised_number: k_means(noised_number.convert('1'))))

Equals = True
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
Equals = False
   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10    10
3    10    10    10    10    10    10    10
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10   

Ну тут вариантов особо не осталось. Поэтому картинки зашумлены в чб, отгадываем в чб.

In [14]:
for number in bw_numbers_area:
    draw_assumption(predict(number, lambda noised_number: k_means(noised_number)))

   0.30  0.35  0.40  0.45  0.50  0.55  0.60
0    10    10    10    10    10    10    10
1    10    10    10    10    10    10    10
2    10    10    10    10    10    10     9
3    10    10    10    10    10    10     9
4    10    10    10    10    10    10    10
5    10    10    10    10    10    10    10
6    10    10    10    10    10    10    10
7    10    10    10    10    10    10    10
8    10    10    10    10    10    10    10
9    10    10    10    10    10    10    10
----------------------------------------------------------------------------------------------------
  0.30 0.35 0.40 0.45 0.50 0.55    0.60
0   {}   {}   {}   {}   {}   {}      {}
1   {}   {}   {}   {}   {}   {}      {}
2   {}   {}   {}   {}   {}   {}  {7: 1}
3   {}   {}   {}   {}   {}   {}  {2: 1}
4   {}   {}   {}   {}   {}   {}      {}
5   {}   {}   {}   {}   {}   {}      {}
6   {}   {}   {}   {}   {}   {}      {}
7   {}   {}   {}   {}   {}   {}      {}
8   {}   {}   {}   {}   {}   {}      {}
9   {}   {}   {

Зашумляем простую белую картинку.

In [15]:
rgb_noise_list = []
grey_noise_list = []
bw_noise_list = []
for i in range(10):
    for noise in noise_indexes:
        rgb_noise_list.append(generate_noise("RGB", (7, 8), 'white', noise,
                                             lambda noised_image, probability: noise_image(noised_image, probability)))
        grey_noise_list.append(generate_noise("L", (7, 8), 'white', noise,
                                              lambda noised_image, probability: noise_image(noised_image, probability)))
        bw_noise_list.append(generate_noise("1", (7, 8), 'white', noise,
                                            lambda noised_image, probability: noise_image(noised_image, probability)))

Сколько посчтаем выбросами

In [16]:
print(f'Выбросов определено: {odin(rgb_noise_list, 5, 35)}. Реальных выбросов: {len(rgb_noise_list)}')
print(f'Выбросов определено: {odin(grey_noise_list, 5, 35)}. Реальных выбросов: {len(grey_noise_list)}')
print(f'Выбросов определено: {odin(bw_noise_list, 5, 35)}. Реальных выбросов: {len(bw_noise_list)}')

Выбросов определено: 66. Реальных выбросов: 70
Выбросов определено: 68. Реальных выбросов: 70
Выбросов определено: 67. Реальных выбросов: 70


Сборная солянка из зашумленых эталонов и зашумленых "пустышек".

In [17]:
mixed_list = []
for n in range(10):
    number = Image.open(f'../../resources/numbers/{7}x{8}/{n}.png').convert('RGB')
    for noise in noise_indexes:
        mixed_list.append(noise_image(number, noise))

mixed_list += rgb_noise_list
print(f'Выбросов определено: {odin(mixed_list, 10, 15)}. '
      f'Размер выборки: {len(mixed_list)}. '
      f'Реальных выбросов: {len(rgb_noise_list)}')

Выбросов определено: 118. Размер выборки: 140. Реальных выбросов: 70
